# AWS Athena - Interactive SQL Analytics on S3

This notebook provides a comprehensive guide to Amazon Athena, a serverless interactive query service that enables you to analyze data directly in Amazon S3 using standard SQL.

---

## Table of Contents

1. **Introduction & Core Concepts** - What is Athena, architecture, pricing
2. **Setup & Prerequisites** - Configuration, boto3 client
3. **Basic Queries** - SELECT, WHERE, GROUP BY, ORDER BY
4. **Advanced Queries** - JOINs, aggregations, window functions
5. **Table Management** - CREATE TABLE, partitioning
6. **Best Practices** - Cost optimization, performance tuning
7. **Complete Data Lake Workflow** - End-to-end pipeline

---

## PHASE 1: INTRODUCTION & CORE CONCEPTS

### What is Amazon Athena?

Amazon Athena is a **serverless, interactive query service** that makes it easy to analyze data directly in Amazon S3 using standard SQL. There's no infrastructure to manage - you simply point to your data in S3, define the schema, and start querying.

**Key Characteristics:**
- **Serverless**: No servers to provision, manage, or scale
- **Pay-per-query**: $5 per TB of data scanned
- **Standard SQL**: ANSI SQL compatible (Presto/Trino engine)
- **Fast**: Parallel query execution across multiple nodes
- **Integrated**: Works with Glue Data Catalog, S3, QuickSight

---

### Athena vs Traditional Data Warehouses

| Aspect | Traditional DW | Amazon Athena |
|--------|----------------|---------------|
| Infrastructure | Managed clusters | Serverless |
| Data Location | Load into warehouse | Query in-place (S3) |
| Cost Model | Per-hour/cluster | Per-query (TB scanned) |
| Setup Time | Hours to days | Minutes |
| Scaling | Manual/Auto-scaling | Automatic |
| Best For | Predictable workloads | Ad-hoc analytics |

### Athena Architecture & Workflow

```
+-----------------------------------------------------------------------------------+
|                            ATHENA WORKFLOW                                        |
+-----------------------------------------------------------------------------------+
|                                                                                   |
|   1. DATA IN S3                        2. SCHEMA IN GLUE CATALOG                  |
|   +------------------+                 +------------------+                       |
|   |   S3 Bucket      |                 |  Data Catalog    |                       |
|   |------------------|                 |------------------|                       |
|   | /raw/            |                 | Database:        |                       |
|   |   sales.csv      |---------------->|   my_data_lake   |                       |
|   |   customers.json |   Crawler or    |                  |                       |
|   | /processed/      |   Manual DDL    | Tables:          |                       |
|   |   sales.parquet  |                 |   - raw_sales    |                       |
|   +------------------+                 |   - customers    |                       |
|                                        +------------------+                       |
|                                                 |                                 |
|                                                 v                                 |
|   3. QUERY WITH SQL                    +------------------+                       |
|   +------------------+                 |     ATHENA       |                       |
|   | SELECT *         |---------------->|------------------|                       |
|   | FROM sales       |                 | - Parse SQL      |                       |
|   | WHERE region =   |                 | - Read schema    |                       |
|   |   'North'        |                 | - Scan S3 data   |                       |
|   +------------------+                 | - Execute query  |                       |
|                                        +------------------+                       |
|                                                 |                                 |
|                                                 v                                 |
|   4. RESULTS                           +------------------+                       |
|   +------------------+                 |  Query Results   |                       |
|   | View in Console  |<----------------|------------------|                       |
|   | Download CSV     |                 | - Displayed      |                       |
|   | Save to S3       |                 | - Saved to S3    |                       |
|   | Use via API      |                 | - Available API  |                       |
|   +------------------+                 +------------------+                       |
|                                                                                   |
+-----------------------------------------------------------------------------------+
```

### Key Features

```
+--------------------------------------------------+
|              ATHENA KEY FEATURES                 |
+--------------------------------------------------+
|                                                  |
|  SERVERLESS                                      |
|  - No infrastructure to manage                   |
|  - Auto-scales based on query complexity         |
|  - Always available                              |
|                                                  |
|  PAY-PER-QUERY                                   |
|  - $5 per TB of data scanned                     |
|  - No charges when not querying                  |
|  - Save money with columnar formats              |
|                                                  |
|  STANDARD SQL                                    |
|  - ANSI SQL compatible                           |
|  - JOINs, window functions, CTEs                 |
|  - Presto/Trino query engine                     |
|                                                  |
|  SUPPORTED FORMATS                               |
|  - CSV, TSV, JSON                                |
|  - Parquet, ORC (columnar - recommended)         |
|  - Avro, Apache logs                             |
|                                                  |
|  INTEGRATIONS                                    |
|  - AWS Glue Data Catalog                         |
|  - Amazon QuickSight                             |
|  - JDBC/ODBC drivers                             |
|                                                  |
+--------------------------------------------------+
```

### Pricing Model

| Component | Cost |
|-----------|------|
| Data scanned | $5.00 per TB |
| Cancelled queries | Charged for data scanned before cancellation |
| DDL statements | Free (CREATE, ALTER, DROP) |
| Failed queries | No charge |

**Cost Optimization Example:**
```
Raw CSV (1 TB)      → Query scans 1 TB    → $5.00
Parquet (100 GB)    → Query scans 100 GB  → $0.50  (10x savings!)
Parquet + Partition → Query scans 10 GB   → $0.05  (100x savings!)
```

## PHASE 2: SETUP & PREREQUISITES

### Console Setup (First Time)

```
+------------------------------------------------+
| ATHENA CONSOLE SETUP                           |
+------------------------------------------------+
| AWS Console -> Athena                          |
|                                                |
| First time setup:                              |
| 1. Settings -> Manage                          |
| 2. Query result location:                      |
|    s3://your-bucket/athena-results/            |
| 3. Save                                        |
|                                                |
| This is where query results are stored.        |
+------------------------------------------------+
```

In [1]:
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import os
import time
import re

load_dotenv()

True

In [2]:
# Environment Configuration
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")

# Athena results location
ATHENA_OUTPUT = f's3://{BUCKET_NAME}/athena-results/'

print(f"Region: {AWS_REGION}")
print(f"Bucket: {BUCKET_NAME}")
print(f"Athena Output: {ATHENA_OUTPUT}")

Region: us-east-2
Bucket: real-learn-s3
Athena Output: s3://real-learn-s3/athena-results/


In [3]:
# Initialize Athena Client
athena_client = boto3.client(
    'athena',
    region_name=AWS_REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)

# Helper function for security
def redact_account_id(text):
    """Redact AWS account ID from text"""
    return re.sub(r':\d{12}:', ':************:', str(text))

print("Athena client initialized")

Athena client initialized


## PHASE 3: RUNNING QUERIES WITH BOTO3

### Function 1: Run Athena Query

Core function to execute SQL queries against Athena.

In [4]:
def run_athena_query(query, database, output_location=None, max_results=100):
    """
    Execute an Athena query and return results
    
    Args:
        query (str): SQL query to execute
        database (str): Glue database name
        output_location (str): S3 path for results (default: ATHENA_OUTPUT)
        max_results (int): Maximum rows to return
    
    Returns:
        list: List of dictionaries with query results
    """
    if output_location is None:
        output_location = ATHENA_OUTPUT
    
    try:
        print(f"Executing query on database '{database}'...")
        print(f"Query: {query[:100]}{'...' if len(query) > 100 else ''}")
        
        # Start query execution
        response = athena_client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={'Database': database},
            ResultConfiguration={'OutputLocation': output_location}
        )
        
        query_execution_id = response['QueryExecutionId']
        print(f"Query ID: {query_execution_id}")
        
        # Wait for query to complete
        while True:
            result = athena_client.get_query_execution(
                QueryExecutionId=query_execution_id
            )
            status = result['QueryExecution']['Status']['State']
            
            if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break
            
            print(f"  Status: {status}")
            time.sleep(1)
        
        if status == 'SUCCEEDED':
            # Get execution stats
            stats = result['QueryExecution']['Statistics']
            data_scanned = stats.get('DataScannedInBytes', 0)
            exec_time = stats.get('TotalExecutionTimeInMillis', 0)
            
            print(f"\nQuery SUCCEEDED")
            print(f"Data scanned: {data_scanned / 1024 / 1024:.2f} MB")
            print(f"Execution time: {exec_time / 1000:.2f} seconds")
            print(f"Estimated cost: ${data_scanned / 1024 / 1024 / 1024 / 1024 * 5:.6f}")
            
            # Get query results
            results = athena_client.get_query_results(
                QueryExecutionId=query_execution_id,
                MaxResults=max_results
            )
            
            # Parse results
            rows = results['ResultSet']['Rows']
            if not rows:
                return []
            
            # First row is headers
            headers = [col.get('VarCharValue', '') for col in rows[0]['Data']]
            data = []
            
            for row in rows[1:]:
                values = [col.get('VarCharValue', '') for col in row['Data']]
                data.append(dict(zip(headers, values)))
            
            print(f"Rows returned: {len(data)}")
            return data
            
        else:
            error = result['QueryExecution']['Status'].get('StateChangeReason', 'Unknown error')
            print(f"Query {status}: {error}")
            return None
            
    except ClientError as e:
        print(f"ERROR: {e}")
        return None

### Function 2: Show Tables

In [5]:
def show_tables(database):
    """
    List all tables in a database
    """
    query = "SHOW TABLES"
    results = run_athena_query(query, database)
    
    if results:
        print("\nTables:")
        for row in results:
            print(f"  - {list(row.values())[0]}")
    
    return results

# Example
show_tables('data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: SHOW TABLES
Query ID: 2384315d-016c-45a6-be4a-89f8d29753ef
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.27 seconds
Estimated cost: $0.000000
Rows returned: 1

Tables:
  - users


[{'raw': 'users'}]

### Function 3: Describe Table

In [6]:
def describe_table(database, table_name):
    """
    Show table schema
    """
    query = f"DESCRIBE {table_name}"
    results = run_athena_query(query, database)
    
    if results:
        print(f"\nSchema for '{table_name}':")
        for row in results:
            col_name = row.get('col_name', '')
            data_type = row.get('data_type', '')
            if col_name and not col_name.startswith('#'):
                print(f"  {col_name}: {data_type}")
    
    return results

# Example
describe_table('data_engineering_db', 'users')

Executing query on database 'data_engineering_db'...
Query: DESCRIBE users
Query ID: fe1ee647-93a2-4eda-a809-b51f4638d73b
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.80 seconds
Estimated cost: $0.000000
Rows returned: 2

Schema for 'users':


[{'id                  \tint                 \t                    ': 'name                \tstring              \t                    '},
 {'id                  \tint                 \t                    ': 'created_at          \ttimestamp           \t                    '}]

## PHASE 4: QUERY EXAMPLES

### Basic SELECT Queries

In [7]:
# Simple SELECT with LIMIT
query = """
SELECT * 
FROM users 
LIMIT 10
"""

results = run_athena_query(query, 'data_engineering_db')
for row in results:
    print(row)

Executing query on database 'data_engineering_db'...
Query: 
SELECT * 
FROM users 
LIMIT 10

Query ID: 12000feb-09a5-4089-8d68-232bfd766074
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.54 seconds
Estimated cost: $0.000000
Rows returned: 10
{'id': '1', 'name': 'Alice Johnson', 'created_at': '2024-01-15 09:30:00.000'}
{'id': '2', 'name': 'Bob Smith', 'created_at': '2024-02-20 14:45:30.000'}
{'id': '3', 'name': 'Carol Williams', 'created_at': '2024-03-10 11:00:00.000'}
{'id': '4', 'name': 'David Brown', 'created_at': '2024-04-05 16:20:15.000'}
{'id': '5', 'name': 'Emma Davis', 'created_at': '2024-05-18 08:15:45.000'}
{'id': '6', 'name': 'Frank Miller', 'created_at': '2024-06-22 13:30:00.000'}
{'id': '7', 'name': 'Grace Wilson', 'created_at': '2024-07-30 10:45:20.000'}
{'id': '8', 'name': 'Henry Taylor', 'created_at': '2024-08-12 15:00:00.000'}
{'id': '9', 'name': 'Ivy Anderson', 'created_at': '2024-09-25 09:10:30.000'}
{'id': '10', 'name': 'Jack Thomas', 'crea

In [9]:
# SELECT with WHERE clause
query = """
SELECT id, name, created_at
FROM users
WHERE name LIKE 'Alice%'
"""

results = run_athena_query(query, 'data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: 
SELECT id, name, created_at
FROM users
WHERE name LIKE 'Alice%'

Query ID: 872b276b-fcf7-4dc0-95ef-e4003f7724eb
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.57 seconds
Estimated cost: $0.000000
Rows returned: 1


### Aggregation Queries

In [10]:
# COUNT and GROUP BY
query = """
SELECT 
    DATE(created_at) as date,
    COUNT(*) as user_count
FROM users
GROUP BY DATE(created_at)
ORDER BY date
"""

results = run_athena_query(query, 'data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: 
SELECT 
    DATE(created_at) as date,
    COUNT(*) as user_count
FROM users
GROUP BY DATE(created_a...
Query ID: 08373576-9231-4d36-ba4e-36129fe6d43a
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.79 seconds
Estimated cost: $0.000000
Rows returned: 10


In [11]:
# GROUP BY with HAVING
query = """
SELECT 
    region,
    COUNT(*) as num_sales,
    SUM(total) as total_revenue,
    ROUND(AVG(total), 2) as avg_sale
FROM raw_sales
GROUP BY region
HAVING SUM(total) > 10000
ORDER BY total_revenue DESC
"""

results = run_athena_query(query, 'data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: 
SELECT 
    region,
    COUNT(*) as num_sales,
    SUM(total) as total_revenue,
    ROUND(AVG(total...
Query ID: fa1774ef-2382-4bf9-9051-c1b807170e08
  Status: QUEUED
Query FAILED: TABLE_NOT_FOUND: line 6:6: Table 'awsdatacatalog.data_engineering_db.raw_sales' does not exist


### JOIN Queries

In [12]:
# JOIN tables
query = """
SELECT 
    c.name,
    c.country,
    COUNT(s.sale_id) as num_purchases,
    SUM(s.total) as total_spent
FROM customers c
JOIN sales s ON c.customer_id = s.customer_id
GROUP BY c.name, c.country
ORDER BY total_spent DESC
LIMIT 10
"""

results = run_athena_query(query, 'data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: 
SELECT 
    c.name,
    c.country,
    COUNT(s.sale_id) as num_purchases,
    SUM(s.total) as total...
Query ID: cc0769de-1f2d-4893-b5d8-57f248f0b370
  Status: QUEUED
Query FAILED: TABLE_NOT_FOUND: line 6:6: Table 'awsdatacatalog.data_engineering_db.customers' does not exist


### Date Functions

In [13]:
# Monthly trend analysis
query = """
SELECT 
    DATE_FORMAT(CAST(sale_date AS DATE), '%Y-%m') as month,
    SUM(total) as monthly_revenue,
    COUNT(*) as num_sales
FROM raw_sales
GROUP BY DATE_FORMAT(CAST(sale_date AS DATE), '%Y-%m')
ORDER BY month
"""

results = run_athena_query(query, 'data_engineering_db')

Executing query on database 'data_engineering_db'...
Query: 
SELECT 
    DATE_FORMAT(CAST(sale_date AS DATE), '%Y-%m') as month,
    SUM(total) as monthly_reven...
Query ID: 7aae9378-e38d-43d1-afeb-e16b883b70c1
  Status: QUEUED
Query FAILED: TABLE_NOT_FOUND: line 5:6: Table 'awsdatacatalog.data_engineering_db.raw_sales' does not exist


## PHASE 5: TABLE MANAGEMENT

### Create External Table (Without Crawler)

In [14]:
def create_external_table(database, table_name, columns, s3_location, 
                          file_format='csv', serde=None):
    """
    Create an external table in Athena
    
    Args:
        database (str): Database name
        table_name (str): Table name
        columns (list): List of (name, type) tuples
        s3_location (str): S3 path to data
        file_format (str): 'csv', 'json', or 'parquet'
    """
    
    # Build column definitions
    col_defs = ',\n    '.join([f"`{name}` {dtype}" for name, dtype in columns])
    
    # Format-specific settings
    if file_format == 'csv':
        row_format = """ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')"""
    elif file_format == 'json':
        row_format = "ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'"
    elif file_format == 'parquet':
        row_format = "STORED AS PARQUET"
    else:
        print(f"ERROR: Unsupported format '{file_format}'")
        return None
    
    query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {table_name} (
    {col_defs}
)
{row_format}
LOCATION '{s3_location}'
TBLPROPERTIES ('skip.header.line.count'='1')
"""
    
    print(f"Creating table '{table_name}'...")
    result = run_athena_query(query, database)
    return result

# Example
columns = [
    ('id', 'INT'),
    ('name', 'STRING'),
    ('created_at', 'TIMESTAMP')
]
create_external_table('data_engineering_db', 'users_manual', columns, f's3://{BUCKET_NAME}/data/users/', 'csv')

Creating table 'users_manual'...
Executing query on database 'data_engineering_db'...
Query: 
CREATE EXTERNAL TABLE IF NOT EXISTS users_manual (
    `id` INT,
    `name` STRING,
    `created_at...
Query ID: 25bdbe92-2d6d-4c01-8179-52211bc7e36f
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.44 seconds
Estimated cost: $0.000000


[]

### Create Partitioned Table

Partitioning improves query performance and reduces costs by scanning only relevant data.

In [15]:
def create_partitioned_table(database, table_name, columns, partition_keys, s3_location):
    """
    Create a partitioned external table
    
    Args:
        database (str): Database name
        table_name (str): Table name
        columns (list): List of (name, type) tuples for data columns
        partition_keys (list): List of (name, type) tuples for partition columns
        s3_location (str): S3 path to data
    """
    
    col_defs = ',\n    '.join([f"`{name}` {dtype}" for name, dtype in columns])
    partition_defs = ', '.join([f"`{name}` {dtype}" for name, dtype in partition_keys])
    
    query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {table_name} (
    {col_defs}
)
PARTITIONED BY ({partition_defs})
STORED AS PARQUET
LOCATION '{s3_location}'
"""
    
    print(f"Creating partitioned table '{table_name}'...")
    result = run_athena_query(query, database)
    return result

# Example
columns = [
    ('sale_id', 'INT'),
    ('product', 'STRING'),
    ('total', 'DOUBLE')
]
partitions = [('region', 'STRING'), ('year', 'INT')]
create_partitioned_table('my_db', 'sales_partitioned', columns, partitions, 's3://bucket/data/')

Creating partitioned table 'sales_partitioned'...
Executing query on database 'my_db'...
Query: 
CREATE EXTERNAL TABLE IF NOT EXISTS sales_partitioned (
    `sale_id` INT,
    `product` STRING,
  ...
Query ID: f5b5b676-1d75-4fa8-a9a3-b5cd24fd2730
  Status: QUEUED
Query FAILED: FAILED: SemanticException [Error 10072]: Database does not exist: my_db


### Load Partitions

In [16]:
def repair_table(database, table_name):
    """
    Discover and load all partitions for a partitioned table
    
    This scans S3 and adds any partitions found to the table metadata.
    """
    query = f"MSCK REPAIR TABLE {table_name}"
    print(f"Repairing table '{table_name}' (loading partitions)...")
    return run_athena_query(query, database)

# Example
repair_table('data_engineering_db', 'sales_partitioned')

Repairing table 'sales_partitioned' (loading partitions)...
Executing query on database 'data_engineering_db'...
Query: MSCK REPAIR TABLE sales_partitioned
Query ID: 60064c74-ed9a-48e3-860e-972eefe94b2d
  Status: QUEUED
  Status: RUNNING

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 1.64 seconds
Estimated cost: $0.000000
Rows returned: 0


[]

### Drop Table

In [17]:
def drop_table(database, table_name):
    """
    Drop an external table (does not delete S3 data)
    """
    query = f"DROP TABLE IF EXISTS {table_name}"
    print(f"Dropping table '{table_name}'...")
    return run_athena_query(query, database)

# Example
drop_table('data_engineering_db', 'temp_table')

Dropping table 'temp_table'...
Executing query on database 'data_engineering_db'...
Query: DROP TABLE IF EXISTS temp_table
Query ID: 1199c746-17c2-43a2-a3c9-b2d494ca87c9
  Status: QUEUED

Query SUCCEEDED
Data scanned: 0.00 MB
Execution time: 0.25 seconds
Estimated cost: $0.000000


[]

## PHASE 6: BEST PRACTICES

### Cost Optimization

```
+------------------------------------------------------------------+
|                    COST OPTIMIZATION                             |
+------------------------------------------------------------------+
|                                                                  |
|  1. USE COLUMNAR FORMATS (Parquet, ORC)                          |
|     - Scan only needed columns                                   |
|     - 5-10x cheaper than CSV/JSON                                |
|     - Better compression                                         |
|                                                                  |
|  2. PARTITION YOUR DATA                                          |
|     - By date, region, category, etc.                            |
|     - Scan only relevant partitions                              |
|     - Use WHERE on partition columns                             |
|                                                                  |
|  3. COMPRESS YOUR DATA                                           |
|     - Gzip, Snappy, ZSTD for Parquet                             |
|     - Reduces data scanned                                       |
|                                                                  |
|  4. USE LIMIT FOR TESTING                                        |
|     - Test queries with LIMIT first                              |
|     - Avoid scanning full tables during development              |
|                                                                  |
|  5. SELECT SPECIFIC COLUMNS                                      |
|     - Avoid SELECT *                                             |
|     - Specify only needed columns                                |
|                                                                  |
+------------------------------------------------------------------+
```

### Performance Tips

```
+------------------------------------------------------------------+
|                    PERFORMANCE TIPS                              |
+------------------------------------------------------------------+
|                                                                  |
|  1. OPTIMIZE JOINs                                               |
|     - Put smaller table on the left side of JOIN                 |
|     - Filter data before joining                                 |
|     - Use appropriate JOIN types                                 |
|                                                                  |
|  2. USE APPROPRIATE DATA TYPES                                   |
|     - INT instead of STRING for numbers                          |
|     - DATE/TIMESTAMP for dates                                   |
|     - Avoid VARCHAR(MAX)                                         |
|                                                                  |
|  3. FILTER EARLY                                                 |
|     - Put WHERE clauses as early as possible                     |
|     - Filter on partition columns first                          |
|                                                                  |
|  4. OPTIMIZE FILE SIZES                                          |
|     - Target 128 MB - 512 MB per file                            |
|     - Avoid many small files                                     |
|     - Avoid very large files (>1 GB)                             |
|                                                                  |
+------------------------------------------------------------------+
```

## PHASE 7: COMPLETE DATA LAKE WORKFLOW

```
+-----------------------------------------------------------------------------------+
|                        COMPLETE DATA LAKE PIPELINE                                |
+-----------------------------------------------------------------------------------+
|                                                                                   |
|  1. INGEST                                                                        |
|     +-------------------+                                                         |
|     | Upload to S3      |  <-- Manual, Lambda, Kinesis Firehose                   |
|     | (raw/ folder)     |                                                         |
|     +-------------------+                                                         |
|              |                                                                    |
|              v                                                                    |
|  2. CATALOG                                                                       |
|     +-------------------+                                                         |
|     | Glue Crawler      |  --> Discovers schema, creates tables                   |
|     | (Data Catalog)    |                                                         |
|     +-------------------+                                                         |
|              |                                                                    |
|              v                                                                    |
|  3. TRANSFORM                                                                     |
|     +-------------------+                                                         |
|     | Glue ETL Job      |  --> Clean, transform, convert to Parquet               |
|     | (processed/)      |                                                         |
|     +-------------------+                                                         |
|              |                                                                    |
|              v                                                                    |
|  4. CATALOG PROCESSED                                                             |
|     +-------------------+                                                         |
|     | Another Crawler   |  --> Update catalog with processed tables               |
|     +-------------------+                                                         |
|              |                                                                    |
|              v                                                                    |
|  5. ANALYZE                                                                       |
|     +-------------------+                                                         |
|     | Athena SQL        |  --> Fast, serverless analytics                         |
|     | Queries           |                                                         |
|     +-------------------+                                                         |
|              |                                                                    |
|              v                                                                    |
|  6. VISUALIZE (Optional)                                                          |
|     +-------------------+                                                         |
|     | QuickSight        |  --> Dashboards, reports                                |
|     +-------------------+                                                         |
|                                                                                   |
|  All serverless, managed, pay-per-use!                                            |
+-----------------------------------------------------------------------------------+
```